# Imports

In [2]:
import json
import emoji
import pandas as pd

# harcoded input variables.
## to be replaced by kwargs* or argv, argc

In [3]:
path = "/Users/sebastian.pizarro/0code/0nexus/aa-dec/data/farmers-protest-tweets-2021-2-4.json"

# Testing input file
## a json per line, splitted by '\n' within the file

In [4]:
from pprint import pprint
raw_data_list = []
with open(path) as fp:
    i = 0 
    for line in fp.readlines():
        line = line.strip()
        raw_data = json.loads(line)
        raw_data_list.append(raw_data)

In [5]:
def reload_df(raw_data_list):
    return pd.DataFrame(raw_data_list)


# Question defs

In [66]:
# 1. Los top 10 tweets más retweeted
# output = tweet_id, count_of_user_instances_id
# logic = id, sum(retweenCount) order by 2 desc limit K
def q1(df):
    from pprint import pprint
    df = df[['id','retweetCount']].reindex()
    df=df.groupby('id')['retweetCount'].agg(MySum='sum')
    df = df.sort_values(by='MySum', ascending=False)
    pprint(df)
    df.info()
    
    
# 2. Los top 10 users en función a la cantidad de tweets que emitieron.
# output = user(username), count_of_tweet_id
# assumption = unique tweets  -- no retweets
# logic = user, count(id) order by 2 desc limit K
def q2(df):
    from pprint import pprint
    df = df[['user','id']]
    df['username'] = df['user'].apply(lambda x : x['username'])
    df = df[['username','id']].reindex()
    df = df.groupby('username')['id'].agg(MyCount='count')
    df = df.sort_values(by='MyCount', ascending=False)
    pprint(df)

# 3. Los top 10 días donde hay más tweets. 
# output = date, count_of_tweet_ids
# assumption = unique tweets  -- no retweets
#  logic = date, count(id) order by 2 desc limit K
def q3(df):
    from pprint import pprint
    df = df[df["date"]!=""]
    df = df[df["date"].notnull()]
    df['date_object'] = pd.to_datetime(df['date'], format="%Y-%m-%dT%H:%M:%S+00:00")
    df['date_day'] =df['date_object'].apply( lambda dt : dt.strftime("%Y-%m-%d"))
    df = df [['date_day','id']]
    print(df.groupby('date_day').count())

# Los top 10 hashtags más usados.
# clean content whitespace, split into words, identify by containing \#
# not necessarily at beggining of word 
# output = hashtag_string, count
# assumption = unique tweets  -- no retweets
def q4(df):
    def get_hashtags_array(text):
        """
        Check if a string contains an emoji.

        Args:
            text (str): The input text to check for emojis.

        Returns:
            bool: True if the input text contains an emoji, False otherwise.
        """
        response = list(text.replace("\n"," ").split(' '))
        d = []
        for e in response:
            i=0
            e=e.strip()
            if '#' not in e:
                continue
            d.append(({"hashtag":e, "count": 1}))
            i=i+1
            if i > 3:
                break
        return d
    df["hashtags"] = df["content"].apply(lambda tweet: get_hashtags_array(tweet))


# 5. Los top 10 emojis más usados.
## use emoji, analyze, retrieve values and store it as array in dataframe
## use this intermediate output and rerout to q4() internal count rountine (mapreduce approach)
## hex_value, count(*) order by 2 desc limit K
def q5(df):
    def has_emoji(text):
        """
        Check if a string contains an emoji.

        Args:
            text (str): The input text to check for emojis.

        Returns:
            bool: True if the input text contains an emoji, False otherwise.
        """
        response = list(emoji.analyze(text.replace("\n","")))
        d = []
        for e in response:
            i=0
            if e is None:
                continue
            d.append(({"emoji":e.value.emoji, "count": 1}))
            i=i+1
            if i > 3:
                break
        
        #print(d)
        return d


    #from pprint import pprint
    df["emojis"] = df["content"].apply(lambda tweet: has_emoji(tweet))
    df["emojis"].sample()
    #df.info()


# 6. Los top 10 users más influyentes en función de lo retweeted de sus tweets.
## output
## assumption = retween count only, not original tweet
## user, count(id) where retweeted_count > 0
def q6(df):
    from pprint import pprint
    df = df.loc[df["retwee"]]
    df = df[['user','id']]

    df['username'] = df['user'].apply(lambda x : x['username'])
    df = df[['username','id']].reindex()
    df = df.groupby('username')['id'].agg(MyCount='count')
    df = df.sort_values(by='MyCount', ascending=False)
    pprint(df)




df=reload_df(raw_data_list)
#q5(df)


In [67]:
#df["emojis"]
q4(df)
df[["content","hashtags"]]

,content,hashtags
0,The world progresses while the Indian police a...,"[{'hashtag': '#ModiDontSellFarmers', 'count': ..."
1,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,"[{'hashtag': '#FarmersProtest', 'count': 1}, {..."
2,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,"[{'hashtag': '#FarmersProtest', 'count': 1}]"
3,@ReallySwara @rohini_sgh watch full video here...,"[{'hashtag': '#farmersprotest', 'count': 1}, {..."
4,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,"[{'hashtag': '#KisanEktaMorcha', 'count': 1}, ..."
...,...,...
117402,#FarmersProtest #KisanAndolan #KisaanMajdoorEk...,"[{'hashtag': '#FarmersProtest', 'count': 1}, {..."
117403,PM मोदी की अपील के बीच संयुक्त किसान मोर्चा का...,"[{'hashtag': '#FarmersProtest', 'count': 1}, {..."
117404,United we stand.\nDivided we fall\n#Mahapancha...,"[{'hashtag': '#MahapanchayatRevolution', 'coun..."
117405,"सिंघु बॉर्डर पर लंबी लड़ाई की तैयारी, किसानों ...","[{'hashtag': '#FarmersProtest', 'count': 1}, {..."


# Stream data kafka producer 
# demo concept @ 60 seconds
# ingest using spark [opt]
# run qX() on window 
# container kafka
# spark-local streaming
# update matplotlib or something else, refresh, etc.